# Welcome! Here you can run all the code directly from your browser. 
## Please NOTE: you will need to have an Alpaca API key generated before running

In [ ]:
#IMPORT DEPENDENCIES
from IPython.display import Image, display
import pandas as pd
!pip install yfinance
import yfinance as yf
import datetime
import numpy as np
from scipy.optimize import minimize
import warnings


In [ ]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
import alpaca_trade_api as tradeapi
!pip install alpaca-trade-api
import datetime as dt
import pytz
!pip install hvplot
import hvplot.pandas

class MCSimulation:
    """
    A Python class for runnning Monte Carlo simulation on portfolio price data. 
    
    ...
    
    Attributes
    ----------
    portfolio_data : pandas.DataFrame
        portfolio dataframe
    weights: list(float)
        portfolio investment breakdown
    nSim: int
        number of samples in simulation
    nTrading: int
        number of trading days to simulate
    simulated_return : pandas.DataFrame
        Simulated data from Monte Carlo
    confidence_interval : pandas.Series
        the 95% confidence intervals for simulated final cumulative returns
        
    """
    
    def __init__(self, portfolio_data, weights="", num_simulation=1000, num_trading_days=252):
        """
        Constructs all the necessary attributes for the MCSimulation object.

        Parameters
        ----------
        portfolio_data: pandas.DataFrame
            DataFrame containing stock price information from Alpaca API
        weights: list(float)
            A list fractions representing percentage of total investment per stock. DEFAULT: Equal distribution
        num_simulation: int
            Number of simulation samples. DEFAULT: 1000 simulation samples
        num_trading_days: int
            Number of trading days to simulate. DEFAULT: 252 days (1 year of business days)
        """
        
        # Check to make sure that all attributes are set
        if not isinstance(portfolio_data, pd.DataFrame):
            raise TypeError("portfolio_data must be a Pandas DataFrame")
            
        # Set weights if empty, otherwise make sure sum of weights equals one.
        if weights == "":
            num_stocks = len(portfolio_data.columns.get_level_values(0).unique())
            weights = [1.0/num_stocks for s in range(0,num_stocks)]
        else:
            if round(sum(weights),2) < .99:
                raise AttributeError("Sum of portfolio weights must equal one.")
        
        # Calculate daily return if not within dataframe
        if not "daily_return" in portfolio_data.columns.get_level_values(1).unique():
            close_df = portfolio_data.xs('close',level=1,axis=1).pct_change()
            tickers = portfolio_data.columns.get_level_values(0).unique()
            column_names = [(x,"daily_return") for x in tickers]
            close_df.columns = pd.MultiIndex.from_tuples(column_names)
            portfolio_data = portfolio_data.merge(close_df,left_index=True,right_index=True).reindex(columns=tickers,level=0)    
        
        # Set class attributes
        self.portfolio_data = portfolio_data
        self.weights = weights
        self.nSim = num_simulation
        self.nTrading = num_trading_days
        self.simulated_return = ""
        
    def calc_cumulative_return(self):
        """
        Calculates the cumulative return of a stock over time using a Monte Carlo simulation (Brownian motion with drift).

        """
        
        # Get closing prices of each stock
        last_prices = self.portfolio_data.xs('close',level=1,axis=1)[-1:].values.tolist()[0]
        
        # Calculate the mean and standard deviation of daily returns for each stock
        daily_returns = self.portfolio_data.xs('daily_return',level=1,axis=1)
        mean_returns = daily_returns.mean().tolist()
        std_returns = daily_returns.std().tolist()
        
        # Initialize empty Dataframe to hold simulated prices
        portfolio_cumulative_returns = pd.DataFrame()
        
        # Run the simulation of projecting stock prices 'nSim' number of times
        for n in range(self.nSim):
        
            #if n % 10 == 0:
                #print(f"Running Monte Carlo simulation number {n}.")
        
            # Create a list of lists to contain the simulated values for each stock
            simvals = [[p] for p in last_prices]
    
            # For each stock in our data:
            for s in range(len(last_prices)):

                # Simulate the returns for each trading day
                for i in range(self.nTrading):
        
                    # Calculate the simulated price using the last price within the list
                    simvals[s].append(simvals[s][-1] * (1 + np.random.normal(mean_returns[s], std_returns[s])))
    
            # Calculate the daily returns of simulated prices
            sim_df = pd.DataFrame(simvals).T.pct_change()
    
            # Use the `dot` function with the weights to multiply weights with each column's simulated daily returns
            sim_df = sim_df.dot(self.weights)
    
            # Calculate the normalized, cumulative return series
            portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()
        
        # Set attribute to use in plotting
        self.simulated_return = portfolio_cumulative_returns
        
        # Calculate 95% confidence intervals for final cumulative returns
        self.confidence_interval = portfolio_cumulative_returns.iloc[-1, :].quantile(q=[0.025, 0.975])
        
        return portfolio_cumulative_returns
    
    def plot_simulation(self):
        """
        Visualizes the simulated stock trajectories using calc_cumulative_return method.

        """ 
        
        # Check to make sure that simulation has run previously. 
        if not isinstance(self.simulated_return,pd.DataFrame):
            self.calc_cumulative_return()
            
        # Use Pandas plot function to plot the return data
        plot_title = f"{self.nSim} Simulations of Cumulative Portfolio Return Trajectories Over the Next {self.nTrading} Trading Days."
        return self.simulated_return.plot(legend=None,title=plot_title)
    
    def plot_distribution(self):
        """
        Visualizes the distribution of cumulative returns simulated using calc_cumulative_return method.
        """
        # Check to make sure that simulation has run previously.
        if not isinstance(self.simulated_return, pd.DataFrame):
            self.calc_cumulative_return()

        simulated_returns_series = self.simulated_return.iloc[-1, :]

        # Use the `hvplot.hist` function to create a probability distribution histogram of simulated ending prices
        # with markings for a 95% confidence interval
        plot_title = f"Distribution of Final Cumulative Returns Across All {self.nSim} Simulations"
        plot = simulated_returns_series.hvplot.hist(bins=10, title=plot_title, density=True)

        # Calculate y-values for horizontal lines
        ci_lower = self.confidence_interval.iloc[0]
        ci_upper = self.confidence_interval.iloc[1]

        # Add horizontal lines for the confidence interval
        line_plot_lower = pd.DataFrame({
            'x': [simulated_returns_series.min(), simulated_returns_series.max()],
            'y': [ci_lower, ci_lower]
        }).hvplot.line(x='x', y='y', color='r')
        plot *= line_plot_lower

        line_plot_upper = pd.DataFrame({
            'x': [simulated_returns_series.min(), simulated_returns_series.max()],
            'y': [ci_upper, ci_upper]
        }).hvplot.line(x='x', y='y', color='r')
        plot *= line_plot_upper

        return plot

    
    def summarize_cumulative_return(self):
        """
        Calculate final summary statistics for Monte Carlo simulated stock data.
        
        """
        
        # Check to make sure that simulation has run previously. 
        if not isinstance(self.simulated_return,pd.DataFrame):
            self.calc_cumulative_return()
            
        metrics = self.simulated_return.iloc[-1].describe()
        ci_series = self.confidence_interval
        ci_series.index = ["95% CI Lower","95% CI Upper"]
        # return metrics.append(ci_series)
        metrics = pd.concat([metrics,ci_series])
        return metrics

In [ ]:
# Suppress the specific PerformanceWarning
warnings.filterwarnings('ignore', message='DataFrame is highly fragmented', category=pd.errors.PerformanceWarning)

In [ ]:
#APIs
# Define the list of stock ticker symbols
tickers = ['AGG', 'BNDX', 'SPY', 'IVW', 'IVE', 'BTC-USD', 'ETH-USD']

# Calculate the start date (three years ago from today)
start_date = datetime.datetime.now() - datetime.timedelta(days=1095)

# Initialize an empty dataframe to store the Close columns for each ticker
new_df = pd.DataFrame()

# Initialize an empty list to store the column names of Close prices for each ticker
ticker_closes = []

In [ ]:
#Past 3 years of data into data frames
# Iterate over each ticker symbol
for ticker in tickers:
    # Fetch historical stock price data for the current ticker
    stock_data = yf.download(ticker, start=start_date, end=datetime.datetime.now())
    
    # Extract the 'Close' column and rename it for the current ticker
    close_column = stock_data[['Close']].rename(columns={'Close': f'{ticker}_Close'})
    
    # Compute the daily returns and insert them to the left of the corresponding ticker's Close column
    daily_returns = close_column.pct_change().rename(columns={f'{ticker}_Close': f'{ticker}_Daily_Return'})
    new_df = pd.concat([daily_returns, new_df], axis=1)
    
    # Append the Close column name for the current ticker to the ticker_closes list
    ticker_closes.append(f'{ticker}_Close')

# Drop rows containing NaNs from the new_df dataframe
new_df.dropna(inplace=True)


In [ ]:
## Feel free to adjust or send feedback if the code could be better or be more precise to what is needed:

span = len(new_df)  # Adjust the span for three years of historical data

# Calculate the annualized exponentially weighted mean return for each asset
annualized_mean_return = new_df.ewm(span=span).mean().iloc[-1] * 252

# Calculate the annualized exponentially weighted standard deviation (risk) for each asset
annualized_std_dev = new_df.ewm(span=span).std().iloc[-1] * np.sqrt(252)

# Display annualized risk and return for each asset
for ticker in tickers:
    print(f'{ticker}:')
    print(f'Annualized Mean Return: {annualized_mean_return[f"{ticker}_Daily_Return"]:.2%}')
    print(f'Annualized Risk (Standard Deviation): {annualized_std_dev[f"{ticker}_Daily_Return"]:.2%}')
    print()

#print(new_df)

In [ ]:
# Create a dictionary with calculated values
assets_data = {
    'Ticker': tickers,
    'Annualized_Mean_Return': [annualized_mean_return[f"{ticker}_Daily_Return"] for ticker in tickers],
    'Annualized_Std_Dev': [annualized_std_dev[f"{ticker}_Daily_Return"] for ticker in tickers]
}

# Convert the dictionary into a DataFrame
individual_assets = pd.DataFrame(assets_data)

# Print the DataFrame
print(individual_assets)

In [ ]:
agg_mean_return = individual_assets['Annualized_Mean_Return']

# Define the objective function to minimize
def objective_function(weights, mean_returns, target_return):
    # Calculate the portfolio return
    portfolio_return = np.dot(mean_returns, weights)
    # Calculate the squared difference between the portfolio return and the target return
    return (portfolio_return - target_return) ** 2

# Define initial guess for weights
initial_guess = np.ones(len(agg_mean_return)) / len(agg_mean_return)  # Equal weights initially

# List of target return percentages
target_returns = [0.05, 0.10, 0.15, .20, .25, .30, 0.35]  # Add more target returns as needed

# Initialize an empty dataframe to store the results
target_weights = pd.DataFrame()

# Define constraint function to ensure weights sum to 1
def constraint(weights):
    return np.sum(weights) - 1

# Loop through each target return
for i, target_return in enumerate(target_returns, start=1):
    # Perform optimization
    result = minimize(objective_function, initial_guess, args=(agg_mean_return, target_return), method='SLSQP', bounds=[(0, 1)] * len(agg_mean_return), constraints={'type': 'eq', 'fun': constraint})

    # Print optimization result
    # print(f"Optimization Result for Target Return {target_return}:")
    # print(result)
    # print("Optimal solution:", result.x)

    # Round the optimal solution to a certain number of decimal places
    rounded_solution = np.round(result.x, decimals=4)

    # Ensure that the rounded weights sum up to 1
    rounded_solution /= np.sum(rounded_solution)

    # Print the rounded and normalized solution
    print("Optimal solution (rounded and normalized):", rounded_solution)

    # Store the rounded and normalized solution in the dataframe
    target_weights[f'{target_return}_weights'] = rounded_solution

# Print the dataframe with the results
# print("Target Weights:")
target_weights


In [ ]:
#CONFIRM PORT RETURNS MATCH TARGETS FROM SOLVER
for i, target_weight_column in enumerate(target_weights.columns):
    
    # Calculate portfolio return using sum product
    portfolio_return = (np.dot(target_weights[target_weight_column], individual_assets['Annualized_Mean_Return'])) * 100

    print("Portfolio Return:", round(portfolio_return, 2))

# target_weights

In [ ]:
# Calculate the correlation matrix
correlation_matrix = new_df.corr()
#print(correlation_matrix)

# Get the column names for the weights based on target return percentages
weight_columns = [col for col in target_weights.columns if col.endswith('_weights')]

# Access the values of these columns and store them in the weights variable
weights = target_weights[weight_columns].values.T

# Define a function to calculate portfolio standard deviation
def calculate_portfolio_std(weights, correlation_matrix, std_deviations):
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(correlation_matrix, weights)) * np.sum(std_deviations ** 2))
    return portfolio_std

# Create a new DataFrame to store portfolio standard deviations
port_stds = pd.DataFrame(index=[f"Portfolio {i}" for i in range(1, 8)], columns=['Portfolio_Std_Dev'])

# Iterate over each row in the DataFrame
for index, row in target_weights.iterrows():
    # Extract weights and standard deviations from the current row
    weights = row[weight_columns].values
for index, row in individual_assets.iterrows():
    std_deviations = row['Annualized_Std_Dev']

    portfolio_std = calculate_portfolio_std(weights, correlation_matrix, std_deviations)
    
    # Add portfolio standard deviation to the port_stds DataFrame
    port_stds.at[f"Portfolio {index + 1}", 'Portfolio_Std_Dev'] = portfolio_std

# Print the DataFrame with portfolio standard deviations
print(port_stds)

In [ ]:
##NOTE were only going to use 10-30% Return Portfolios for analysis

for i, target_weight_column in enumerate(target_weights.columns[1:-1]):
    # Calculate portfolio return using sum product
    portfolio_return = (np.dot(target_weights[target_weight_column], individual_assets['Annualized_Mean_Return'])) * 100
    
    # Calculate portfolio Sharpe ratio
    portfolio_std_dev = port_stds.iloc[i + 1]['Portfolio_Std_Dev']
    portfolio_sharpe = portfolio_return / (portfolio_std_dev * 100)
    
    # Print portfolio return and Sharpe ratio
    print(f"Portfolio Return: {round(portfolio_return, 2)}%")
    print("Portfolio Sharpe:", round(portfolio_sharpe, 2))
    print('--------')


In [ ]:
monte_df = pd.DataFrame()

# Get the column names for the weights based on target return percentages
weight_columns = [col for col in target_weights.columns if col.endswith('_weights')]

# Access the values of these columns and store them in the weights variable
weights = target_weights[weight_columns].values.T
# print(sum(list(weights[0])))
# print(sum(list(weights[1])))

# Iterate over each ticker symbol
for ticker in tickers:
    # Fetch historical stock price data for the current ticker
    stock_data = yf.download(ticker, start=start_date, end=datetime.datetime.now())
    
    stock_data = stock_data[['Close']].rename(columns={'Close': 'close'})
  
    existing_columns = stock_data.columns

    # Create a MultiIndex with a single top level and the existing column names as the second level
    multi_index = pd.MultiIndex.from_product([[ticker], existing_columns])

    # Set the MultiIndex as the columns of the DataFrame
    stock_data.columns = multi_index

    monte_df = pd.concat([stock_data, monte_df], axis=1)

# Drop rows containing NaNs from the new_df dataframe
monte_df.dropna(inplace=True)

monte_df

## Please be aware, the following simulation cells can take a while. You can adjust the number of simulations run as needed

In [ ]:
# Create an empty dictionary to store MCSimulation instances
MC_ports = {}
num_sims = 100
forecast_years = 5
portfolios = len(list(weights[1:-1]))

# Iterate over the range of portfolios
for i in range(1, portfolios+1):
    # Generate the weights dynamically based on the loop index
    weights_i = list(weights[i])

    # Create an instance of MCSimulation and store it in the dictionary
    MC_ports[f'MC_port_{i}'] = MCSimulation(monte_df, weights=weights_i, num_simulation=num_sims, num_trading_days=252*forecast_years)

    # Print the simulation input data for the current MCSimulation instance
    # print(f"MC_port_{i} simulation input data:")
    # print(MC_ports[f'MC_port_{i}'].portfolio_data.head())

    line_plot = MC_ports[f'MC_port_{i}'].plot_simulation()
    line_plot.get_figure().savefig(f'MC_port_{i}_{forecast_years}year_sim_plot.png', bbox_inches='tight')
    


In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
portfolio_num = 1

for key in MC_ports:
    summary_metrics = MC_ports[key].simulated_return.iloc[-1].describe()
    ci_series_port = MC_ports[key].confidence_interval
    ci_series_port.index = ["95% CI Lower", "95% CI Upper"]

    # Concatenate metrics and confidence interval series into a DataFrame
    summary_df_port = pd.concat([summary_metrics, ci_series_port], axis=0)

    # Print summary statistics with confidence intervals
    print(f"Metrics for Portfolio {portfolio_num}:")
    print('')
    print(summary_df_port)
    

    # Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $15,000 investments stocks
    initial_investment = 15000
    lower_bound_port = round((summary_df_port['95% CI Lower'] * initial_investment), 2)
    upper_bound_port = round((summary_df_port['95% CI Upper'] * initial_investment), 2)

    # Print results
    print('')
    print(f"There is a 95% chance that an initial investment of ${initial_investment}")
    print(f"in Portofolio {portfolio_num} over the next {forecast_years} years will end within the range of ${lower_bound_port} and ${upper_bound_port}.")

    sharpe_port = summary_df_port['mean']/summary_df_port['std']
    print(f"Portfolio {portfolio_num} Sharpe: {round(sharpe_port, 2)}, you make {round(sharpe_port,2)} units of return for every 1 unit of risk")

    # set equal portfolio metrics
    risk_reward_port = round((upper_bound_port- initial_investment)/abs(lower_bound_port - initial_investment), 2)
    sharpe_port = summary_df_port['mean']/summary_df_port['std']
    risk_adjusted_efficiency = round((risk_reward_port / sharpe_port), 2)
    print("--------")
    print(f"Initial Investment: ${initial_investment}")
    print('')
    print(f'Upper and Lower Bounds of Portfolio {portfolio_num}: ${upper_bound_port} and ${lower_bound_port}')
    print(f"Portfolio {portfolio_num} Max Profit: ${round((upper_bound_port - initial_investment), 2)}")
    print(f"Portfolio {portfolio_num} Max Loss: $ {round((lower_bound_port - initial_investment), 2)}")
    print(f"Portfolio {portfolio_num} Risk-Reward Ratio: {risk_reward_port}")
    print(f"Portfolio {portfolio_num} Simulation Sharpe: {round(sharpe_port, 2)}")
    print(f"Portfolio {portfolio_num} Risk-Adjusted Efficiency: {risk_adjusted_efficiency}")


    print('------------------------')
    print('------------------------')
    print('------------------------')

    portfolio_num += 1

In [ ]:
sorted_risk = port_stds[1:-1]
sorted_risk = sorted_risk.sort_values(by='Portfolio_Std_Dev', ascending=True)
sorted_risk

In [ ]:
port_dict = {'port_1': (.10, 3.19), 'port_2': (.15, 1.8), 'port_3': (.20, 1.97), 'port_4': (.25, 3.4), 'port_5': (.30, 1.12)}


# Prompt the user to input desired level of return
investment_amount = int(input("Enter your estimated investment amount: "))
desired_return = float(input("Enter your desired level of return (.10, .15, .20, .25, .30): "))

# Prompt the user to input their comfort level with risk
comfort_level_risk = int(input("Enter your comfort level with risk (on a scale of 1 to 5): "))
portfolio_std_dev = sorted_risk.iloc[comfort_level_risk - 1]['Portfolio_Std_Dev']

# Print the selected portfolio standard deviation
print("Portfolio Standard Deviation for Comfort Level:", portfolio_std_dev)
user_sharpe = desired_return/portfolio_std_dev

# Print the inputs provided by the user
print("Expected Investment Amount:", investment_amount)
print("Desired level of return:", desired_return)
print("Comfort level with risk:", round(portfolio_std_dev, 2))
print("Computed Sharpe:", round(user_sharpe, 2))


In [ ]:
# Find portfolios with the same level of return and a smaller Sharpe
better_portfolios = {}
for portfolio, (ret, sharpe) in port_dict.items():
    if ret == desired_return and sharpe > user_sharpe:
        better_portfolios[portfolio] = sharpe

# If no portfolios found, find portfolios with a higher level of return and a better Sharpe
if not better_portfolios:
    for portfolio, (ret, sharpe) in port_dict.items():
        if ret > desired_return and sharpe > user_sharpe:
            better_portfolios[portfolio] = sharpe
print('----------------------')
print("Portfolios with the same level of return or higher and equal or better Sharpe:")
for portfolio, sharpe in better_portfolios.items():
    print(f"{portfolio}: Historical Return = {port_dict[portfolio][0]}, Historical Sharpe = {sharpe}")

    summary_metrics = MC_ports[f'MC_{portfolio}'].simulated_return.iloc[-1].describe()
    ci_series_port = MC_ports[key].confidence_interval
    ci_series_port.index = ["95% CI Lower", "95% CI Upper"]

    # Concatenate metrics and confidence interval series into a DataFrame
    summary_df_port = pd.concat([summary_metrics, ci_series_port], axis=0)

    # Print summary statistics with confidence intervals
    print(f"Simulated Metrics for Portfolio:")
    print('')
    print(summary_df_port)
    

    # Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $15,000 investments stocks
    initial_investment = investment_amount
    lower_bound_port = round((summary_df_port['95% CI Lower'] * initial_investment), 2)
    upper_bound_port = round((summary_df_port['95% CI Upper'] * initial_investment), 2)

    # Print results
    print('')
    print(f"There is a 95% chance that an initial investment of ${initial_investment}")
    print(f"in this Portofolio over the next {forecast_years} years will end within the range of ${lower_bound_port} and ${upper_bound_port}.")
    
    plot_dist = MC_ports[f'MC_{portfolio}'].plot_distribution()
    display(plot_dist)

    # File path of the PNG file
    file_path = f"MC_{portfolio}_{forecast_years}year_sim_plot.png"

    # Display the PNG file
    display(Image(filename=file_path))

    